Import stuff

In [1]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform

Read from the csv and rename the specific columns

In [111]:
file = 'np_data.csv'
df = pd.read_csv(file, sep = ",", usecols = ['CounterID_ASTA', 'StartTime', 'EndTime', 'Visits', 'ASTA_Counters.NationalParkCode', 'PAVE_Counters.CoordinateNorth', 'PAVE_Counters.CoordinateEast'])
df.columns = ['id', 'start', 'end', 'visits', 'np_code', 'loc_north', 'loc_east']
df

,id,start,end,visits,np_code,loc_north,loc_east
0,1099,19/07/2013 15:00,19/07/2013 16:00,0,34361,7500782.0,381166.0
1,1099,19/07/2013 19:00,19/07/2013 20:00,0,34361,7500782.0,381166.0
2,1099,19/07/2013 20:00,19/07/2013 21:00,0,34361,7500782.0,381166.0
3,1099,19/07/2013 21:00,19/07/2013 22:00,0,34361,7500782.0,381166.0
4,1099,19/07/2013 22:00,19/07/2013 23:00,0,34361,7500782.0,381166.0
...,...,...,...,...,...,...,...
565553,1277,25/02/2019 18:00,25/02/2019 19:00,28,34361,7503667.0,380373.0
565554,1277,25/02/2019 20:00,25/02/2019 21:00,3,34361,7503667.0,380373.0
565555,1277,26/02/2019 10:00,28/02/2019 23:59,49,34361,7503667.0,380373.0
565556,1277,1/03/2019 00:01,31/03/2019 23:59,1250,34361,7503667.0,380373.0


National park ids:
852 = Nuuksio National Park
34361 = Pallas-Yllästunturi National Park
Drop the P-Y National Park

In [113]:
df = df.drop(df[df.np_code == 34361].index)
df['start'] = pd.to_datetime(df['start'])
df['end']  = pd.to_datetime(df['end'])

In [114]:
df2 = df
df2

,id,start,end,visits,np_code,loc_north,loc_east
194738,1043,2015-01-01 00:00:00,2015-01-01 01:00:00,0,852,6687729.0,364641.0
194739,1043,2015-01-01 01:00:00,2015-01-01 02:00:00,0,852,6687729.0,364641.0
194740,1043,2015-01-01 02:00:00,2015-01-01 03:00:00,0,852,6687729.0,364641.0
194741,1043,2015-01-01 03:00:00,2015-01-01 04:00:00,0,852,6687729.0,364641.0
194742,1043,2015-01-01 04:00:00,2015-01-01 05:00:00,0,852,6687729.0,364641.0
...,...,...,...,...,...,...,...
508769,912,2019-03-10 19:00:00,2019-03-10 20:00:00,3,852,6690448.0,361700.0
508770,912,2019-04-10 07:00:00,2019-04-10 08:00:00,3,852,6690448.0,361700.0
508771,912,2019-04-10 14:00:00,2019-04-10 15:00:00,3,852,6690448.0,361700.0
508772,912,2019-06-10 09:00:00,2019-06-10 10:00:00,3,852,6690448.0,361700.0


In [115]:
uniqueCounters = df2['id'].unique()
print(uniqueCounters)

uniqueParkCodes =  df2['np_code'].unique()
print(uniqueParkCodes)

[1043 1050 1225 1246  912  922  871  149 1122]
[852]


Clear everything before 2010

In [152]:
df2 = df2.drop(df2[df2.end < '1/01/2019 00:00'].index)
df2['duration'] = df.apply(lambda row: row.end-row.start, axis=1)
df3 = df2

ids = df3['id'].unique()
for i in range(len(ids)):
    df3 = df3.replace(ids[i],i)
df3

,id,start,end,visits,np_code,loc_north,loc_east,duration
216845,0,2019-01-01 00:00:00,2019-01-01 01:00:00,0,852,6687729.0,364641.0,01:00:00
216846,0,2019-01-01 01:00:00,2019-01-01 02:00:00,0,852,6687729.0,364641.0,01:00:00
216847,0,2019-01-01 02:00:00,2019-01-01 03:00:00,0,852,6687729.0,364641.0,01:00:00
216848,0,2019-01-01 03:00:00,2019-01-01 04:00:00,0,852,6687729.0,364641.0,01:00:00
216849,0,2019-01-01 04:00:00,2019-01-01 05:00:00,0,852,6687729.0,364641.0,01:00:00
...,...,...,...,...,...,...,...,...
508769,4,2019-03-10 19:00:00,2019-03-10 20:00:00,3,852,6690448.0,361700.0,01:00:00
508770,4,2019-04-10 07:00:00,2019-04-10 08:00:00,3,852,6690448.0,361700.0,01:00:00
508771,4,2019-04-10 14:00:00,2019-04-10 15:00:00,3,852,6690448.0,361700.0,01:00:00
508772,4,2019-06-10 09:00:00,2019-06-10 10:00:00,3,852,6690448.0,361700.0,01:00:00


In [161]:
counter_data = []
for id in range (0,len(uniqueCounters)):
    #print(df3.id)
    #print(df3[(df3.duration == '01:00:00') & (df3.id == id)].visits.sum())
    #print(df3[(df3.duration > '02:00:00') & (df3.id == id)].visits.sum())
    #print(df3[df3.id == id].visits.sum())
    counter_data.append([id, df3[df3.id == id].visits.sum(),  df3[df3.id == id].loc_north.max(), df3[df3.id == id].loc_east.max(), 0, 0])
    #print(df3[df3.id == 1043].visits.sum())
    
counter_data

[[0, 22436, 6687729.0, 364641.0, 0, 0],
 [1, 8207, 6684826.0, 359733.0, 0, 0],
 [2, 3854, 6687332.0, 359420.0, 0, 0],
 [3, 10127, 6685472.0, 362093.0, 0, 0],
 [4, 16374, 6690448.0, 361700.0, 0, 0],
 [5, 2218, 6690877.0, 361577.0, 0, 0],
 [6, 8322, 6688995.0, 358986.0, 0, 0],
 [7, 7860, 6681442.0, 372603.0, 0, 0],
 [8, 0, nan, nan, 0, 0]]

In [162]:
def convertCoordinates(x1, y1):
    inProj  = Proj("+init=EPSG:3067")
    outProj = Proj("+init=EPSG:4326")
    #x1,y1 =  (381166, 7500782)
    lat,long = transform(inProj,outProj,x1,y1)
    place = [long,lat]
    return place

convertCoordinates(364641, 6687729)

[60.303525451006834, 24.550391008969527]

In [163]:
for counter in counter_data:
    x1 = counter[2]
    y1 = counter[3]
    #print(x1, y1)
    counter[4], counter[5] = convertCoordinates(y1, x1)
counter_data

[[0, 22436, 6687729.0, 364641.0, 60.303525451006834, 24.550391008969527],
 [1, 8207, 6684826.0, 359733.0, 60.27581963490531, 24.463689284278516],
 [2, 3854, 6687332.0, 359420.0, 60.29819039594651, 24.45628974906327],
 [3, 10127, 6685472.0, 362093.0, 60.28242199633351, 24.505873789599413],
 [4, 16374, 6690448.0, 361700.0, 60.326924542956334, 24.495358359272675],
 [5, 2218, 6690877.0, 361577.0, 60.33073081360451, 24.492838046488888],
 [6, 8322, 6688995.0, 358986.0, 60.31295694299057, 24.447281862291263],
 [7, 7860, 6681442.0, 372603.0, 60.24969716104745, 24.698307563459544],
 [8, 0, nan, nan, inf, inf]]

In [164]:
del counter_data[8]
counter_data

for counter in counter_data:
    del counter[2:4]
counter_data

[[0, 22436, 60.303525451006834, 24.550391008969527],
 [1, 8207, 60.27581963490531, 24.463689284278516],
 [2, 3854, 60.29819039594651, 24.45628974906327],
 [3, 10127, 60.28242199633351, 24.505873789599413],
 [4, 16374, 60.326924542956334, 24.495358359272675],
 [5, 2218, 60.33073081360451, 24.492838046488888],
 [6, 8322, 60.31295694299057, 24.447281862291263],
 [7, 7860, 60.24969716104745, 24.698307563459544]]

In [165]:
header = "id, count, latitude, longitude"
np.savetxt("counters_simple.csv", counter_data, delimiter=",", fmt='%s', header=header, comments='')